In [3]:
from importlib import reload

import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)


sys.path.append(parent_dir)

import src

from src.utils import load_MNIST_data
from src.model import Simple_Model
from src.train import train_model, tune_model
from src.inference import evaluate_overall, evaluate_classwise, confusion_matrix_plot, plot_roc_curves
reload(src.train)
reload(src.model)
reload(src.utils)
reload(src.inference)

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt



In [7]:
train_dloader, val_dloader, test_dloader = load_MNIST_data(batch_size = 128, num_workers=4) # try increase batch size to 128

In [3]:
hyperparam_space = {
    "lr": {"type": "float", "low": 1e-5, "high": 1e-2, "log": True},
    "weight_decay": {"type": "float", "low": 1e-5, "high": 1e-2, "log": True},
    "out_channels": {"type": "int", "low": 4, "high": 16},
    "kernel_size": {"type": "int", "low": 3, "high": 7}
}

In [10]:
device = torch.device("mps")

In [6]:

tune_model(Simple_Model, train_dloader, val_dloader, model_name="Simple_CNN_v3", device=device, n_trials=50, hyperparam_space=hyperparam_space, epochs=3, project_name="Fashion-MNIST-CNN_v3")

NameError: name 'train_dloader' is not defined

In [8]:

# Load optimal parameters first
import json

with open("/Users/leonwindett/VS_CODE/Projects/FASHION-MNIST-CNN/saved_models/Simple_CNN_v3/study_params.json", "r") as f:
    optimal_params = json.load(f)

print("Optimal Hyperparameters found: ", optimal_params)

# Create model instance with optimal parameters
# Initialize model with optimal parameters
CNN_Model = Simple_Model(
    out_channels=optimal_params["out_channels"],
    kernel_size=optimal_params["kernel_size"]
)

# Load the pre-trained weights
CNN_Model.load_state_dict(torch.load(os.path.join(parent_dir, "saved_models/Simple_CNN_v3/best_model.pth")))
optimiser = torch.optim.AdamW(CNN_Model.parameters(), lr=optimal_params["lr"], weight_decay=optimal_params["weight_decay"])
loss_fcn = nn.CrossEntropyLoss()




Optimal Hyperparameters found:  {'lr': 0.004717057291945161, 'weight_decay': 0.0001588709253642451, 'out_channels': 16, 'kernel_size': 3}


In [11]:
best_val_acc = 0
patience = 5
counter = 0
for epoch in range(30):
    train_loss, val_loss, val_acc = train_model(CNN_Model, optimiser, loss_fcn, train_dloader, val_dloader, epoch, device=device, print_training=True)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print("Early stopping triggered")
        break


[Epoch  1] Training accuracy: 90.75%, Validation accuracy: 89.31%


[Epoch  2] Training accuracy: 91.84%, Validation accuracy: 89.02%


[Epoch  3] Training accuracy: 92.13%, Validation accuracy: 89.68%


[Epoch  4] Training accuracy: 92.75%, Validation accuracy: 89.43%


[Epoch  5] Training accuracy: 93.02%, Validation accuracy: 90.06%


[Epoch  6] Training accuracy: 93.26%, Validation accuracy: 89.99%


[Epoch  7] Training accuracy: 93.58%, Validation accuracy: 89.28%


[Epoch  8] Training accuracy: 93.84%, Validation accuracy: 89.36%


[Epoch  9] Training accuracy: 94.16%, Validation accuracy: 89.58%


[Epoch 10] Training accuracy: 94.19%, Validation accuracy: 89.90%
Early stopping triggered


Model began to overfit

In [12]:
test_acc = evaluate_overall(CNN_Model, test_dloader, device)
print(f"Test accuracy: {test_acc:.4f}")

Test accuracy: 0.8959
